# Pyrat Deep Q-Learning Processing

## Setup Environment

Required libraries for PyRat Q-Learning

In [1]:
import json
import numpy as np
import time
import random
import pickle
from tqdm import tqdm
from AIs import manh, numpy_rl_reload
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

### The game.py file describes the simulation environment, including the generation of reward and the observation that is fed to the agent.
import game

### The rl.py file describes the reinforcement learning procedure, including Q-learning, Experience replay, and a pytorch model to learn the Q-function.
### SGD is used to approximate the Q-function.
import rl

Libraries for training the Convolutional Neural Network

In [2]:
# Import libraries

import torch.nn.functional as F
import inspect

# Personal libraries

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

This is a very unknown but cool library that can help you build a neural network.

It helps you **calculate the shape of the tensor outputs** of network operations.

[Tensorshape Library Documentation](https://pypi.org/project/torchshape/0.0.8/#description)

In [3]:
import subprocess # For importing missing libraries real-time
try:
    from torchshape import tensorshape
except:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torchshape'])
    from torchshape import tensorshape

Define our **device** as the first visible CUDA device if we have CUDA available:

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device '+ str(device))

Using device cuda


## PyRat Game Specifications

Details of the game:

1️⃣ The **opponent** plays using a deterministic strategy: a **greedy algorithm** always targetting the closest piece of cheese as next target. - The distance to pieces of cheese is calculated using Manhattan Distance (= L1 distance). The code is in AIs/manh.py.

2️⃣ The maze does **not** include **walls** (option -d 0)

3️⃣ The maze does **not** include **mud** (option -md 0)

4️⃣The **dimension** of the maze is **21 x 15** (default parameter)

5️⃣The number of **pieces of cheese** is **40** (option -p 40)

6️⃣ The **maze** is non **symmetric** (option --nonsymmetric)

You can therefore run a 1000 game test simulations using the following command:

<pre>python pyrat.py -d 0 -md 0 -p 40 -x 21 -y 15 --rat AIs/manh.py --python AIs/YOURAIHERE --nonsymmetric --nodrawing --tests 1000 --synchronous</pre>

Furthermore, you can run a visual game simulation with a command following the next structure:

<pre>python pyrat.py -p 40 -x 21 -y 15 -d 0 -md 0 --rat AIs/manh.py --python AIs/YOURAIHERE --nonsymmetric</pre>

## Train a model to approximate the Q-function

Definitions :
- An iteration of training is called an **Epoch**. It correspond to a full play of a PyRat game. 
- An **experience** is a set of  vectors < s, a, r, s’ > describing the consequence of being in state s, doing action a, receiving reward r, and ending up in state s'.
- Look at the file rl.py to see how the **experience replay buffer** is implemented. 
- A **batch** is a set of experiences we use for training during one epoch. We draw batches from the experience replay buffer.

### Create the simulated game environment

Set the parameters for the Pyrat game simulated environment.

In [5]:
width = 21  # Size of the playing field
height = 15  # Size of the playing field
cheeses = 40  # Number of cheeses in the game
opponent = manh  # AI used for the opponent

Create the Pyrat simulated environment.

In [6]:
env = game.PyRat(width=width, height=height, opponent=opponent, cheeses=cheeses)

Show the **shape of an observation**.

In [7]:
test_observation = torch.FloatTensor(env.observe())
test_observation.shape

torch.Size([1, 29, 41, 1])

We have to be careful with this default shape since convolutional layers expect as inputs tensors in the form of:

<pre>(batch size, number of channels, height, width)</pre>

The environment throws the tensor in the shape, which is **WRONG**:

<pre>(batch size, height, width, number of channels)</pre>

We can transform the tensor in the following way:

In [8]:
test_observation = test_observation.permute(0, 3, 1, 2)
test_observation.shape

torch.Size([1, 1, 29, 41])

### Create the experience replay buffer

Set the parameters for the experience replay buffer.

In [9]:
max_memory = 1000  # Maximum number of experiences we are storing
discount_factor=.97 # Discount factor for future rewards

Create the experience replay buffer.

In [10]:
exp_replay = rl.ExperienceReplay(max_memory=max_memory, discount=discount_factor)
exp_replay.discount

0.97

### Q-Function Approximation Model Topologies

#### Model 1

**Simple regressor** to predict the Q-values. Base Topology used and tested in course laboratory.

In [11]:
class MultiRegressor1FC(nn.Module):
    def __init__(self, x_example, number_of_channels=1, number_of_regressors=4):
        super(MultiRegressor1FC, self).__init__()
        in_features = x_example.reshape(-1).shape[0]
        self.nb_channels = number_of_channels
        self.linear = nn.Linear(in_features, number_of_regressors)
    
    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        return self.linear(x)

    def load(self):
        if self.nb_channels == 1:
            self.load_state_dict(torch.load('save_rl/weights_ANN1FC_1channel.pt'))
        else:
            self.load_state_dict(torch.load('save_rl/weights_ANN1FC_2channel.pt'))

    def save(self):
        if self.nb_channels == 1:
            torch.save(self.state_dict(), 'save_rl/weights_ANN1FC_1channel.pt')
        else:
            torch.save(self.state_dict(), 'save_rl/weights_ANN1FC_2channel.pt')

#### Model 2

**1 hidden layer regressor network** to predict the Q-values.

In [12]:
class MultiRegressor2FC(nn.Module):
    def __init__(self, x_example, number_of_channels=1, number_of_regressors=4):
        super(MultiRegressor2FC, self).__init__()
        in_features = x_example.reshape(-1).shape[0]
        self.nb_channels = number_of_channels
        self.fc1 = nn.Linear(in_features, 16)
        self.selu = nn.SELU()
        self.linear = nn.Linear(16, number_of_regressors)
    
    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.selu(x)
        return self.linear(x)

    def load(self):
        if self.nb_channels == 1:
            self.load_state_dict(torch.load('save_rl/weights_ANN2FC_1channel.pt'))
        else:
            self.load_state_dict(torch.load('save_rl/weights_ANN2FC_2channel.pt'))

    def save(self):
        if self.nb_channels == 1:
            torch.save(self.state_dict(), 'save_rl/weights_ANN2FC_1channel.pt')
        else:
            torch.save(self.state_dict(), 'save_rl/weights_ANN2FC_2channel.pt')

#### Model 3

A **CNN multi-regressor** integrating **1 fully connected layer**. Expects **1 or 2 channels** as input.

In [13]:
class MultiRegressorCNN1FC(nn.Module):
    def __init__(self, number_of_channels=1):
        super().__init__()
        self.nb_channels = number_of_channels
        self.conv1 = nn.Conv2d(number_of_channels, 16, kernel_size=3) # output_shape = (1, 16, 27, 39)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 16, 13, 19)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3) # output_shape = (1, 32, 11, 17)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 32, 5, 8)
        self.fc = nn.Linear(32 * 5 * 8, 4) 
        
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.reshape(x.shape[0],-1) # output_shape = (1280)
        x = self.fc(x)
        return x
    
    def load(self):
        if self.nb_channels == 1:
            self.load_state_dict(torch.load('save_rl/weights_CNN1FC_1channel.pt'))
        else:
            self.load_state_dict(torch.load('save_rl/weights_CNN1FC_2channel.pt'))

    def save(self):
        if self.nb_channels == 1:
            torch.save(self.state_dict(), 'save_rl/weights_CNN1FC_1channel.pt')
        else:
            torch.save(self.state_dict(), 'save_rl/weights_CNN1FC_2channel.pt')

Sanity check to confirm the dimensions of the tensors after each convolutional layer operations.

In [14]:
# Input shape which is the size of the canvas

x_shape = (1, 1, 29, 41)

# Default not passed parameters for Conv2d
# stride=(1,1), padding=(0,0), dilation=(1,1), groups=1

# First convolution operation
op = nn.Conv2d(1, 16, kernel_size=3)
x_shape = tensorshape(op, x_shape)
print(f'Shape after first Conv2d: {x_shape}')

# First maxpool operation
op = nn.MaxPool2d(kernel_size=2)
x_shape = tensorshape(op, x_shape)
print(f'Shape after first MaxPool2d: {x_shape}')

# Second convolution operation
op = nn.Conv2d(16, 32, kernel_size=3)
x_shape = tensorshape(op, x_shape)
print(f'Shape after second Conv2d: {x_shape}')

# Second maxpool operation
op = nn.MaxPool2d(kernel_size=2)
x_shape = tensorshape(op, x_shape)
print(f'Shape after second MaxPool2d: {x_shape}')

Shape after first Conv2d: (1, 16, 27, 39)
Shape after first MaxPool2d: (1, 16, 13, 19)
Shape after second Conv2d: (1, 32, 11, 17)
Shape after second MaxPool2d: (1, 32, 5, 8)


Check that this model fits well the data with a small sanity check.

In [15]:
# Create a test instance of the model
test_model = MultiRegressorCNN1FC(env.observe().shape[3])

# Get a sample observation of the game environment
test_input_tensor = torch.FloatTensor(env.observe())
print(f'Shape of the raw input tensor: {test_input_tensor.shape}')

# Get an output given the sample input and an untrained model just to validate teh correct output size
test_output_tensor = test_model(test_input_tensor)
print(f'Shape of the output tensor: {test_output_tensor.shape}')

Shape of the raw input tensor: torch.Size([1, 29, 41, 1])
Shape of the output tensor: torch.Size([1, 4])


#### Model 4

A **CNN multi-regressor** integrating **2 fully connected layers**. Expects **1 or 2 channels** as input.

In [16]:
class MultiRegressorCNN2FC(nn.Module):
    def __init__(self, number_of_channels=1):
        super().__init__()
        self.nb_channels = number_of_channels
        self.conv1 = nn.Conv2d(number_of_channels, 16, kernel_size=3) # output_shape = (1, 16, 27, 39)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 16, 13, 19)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3) # output_shape = (1, 32, 11, 17)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 32, 5, 8)
        self.fc1 = nn.Linear(32 * 5 * 8, 16)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(16, 4)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        #x = self.dropout(x)
        
        x = x.reshape(x.shape[0],-1) # output_shape = (1280)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    def load(self):
        if self.nb_channels == 1:
            self.load_state_dict(torch.load('save_rl/weights_CNN2FC_1channel.pt'))
        else:
            self.load_state_dict(torch.load('save_rl/weights_CNN2FC_2channel.pt'))

    def save(self):
        if self.nb_channels == 1:
            torch.save(self.state_dict(), 'save_rl/weights_CNN2FC_1channel.pt')
        else:
            torch.save(self.state_dict(), 'save_rl/weights_CNN2FC_2channel.pt')

Check that this model fits well the data with a small sanity check.

In [17]:
# Create a test instance of the model
test_model = MultiRegressorCNN2FC(env.observe().shape[3])

# Get a sample observation of the game environment
test_input_tensor = torch.FloatTensor(env.observe())
print(f'Shape of the raw input tensor: {test_input_tensor.shape}')

# Get an output given the sample input and an untrained model just to validate teh correct output size
test_output_tensor = test_model(test_input_tensor)
print(f'Shape of the output tensor: {test_output_tensor.shape}')

Shape of the raw input tensor: torch.Size([1, 29, 41, 1])
Shape of the output tensor: torch.Size([1, 4])


#### Model 5

A **CNN multi-regressor** integrating **3 fully connected layers**. Expects **1 or 2 channels** as input.

In [18]:
class MultiRegressorCNN3FC(nn.Module):
    def __init__(self, number_of_channels=1):
        super().__init__()
        self.nb_channels = number_of_channels
        self.conv1 = nn.Conv2d(number_of_channels, 16, kernel_size=3) # output_shape = (1, 16, 27, 39)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 16, 13, 19)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3) # output_shape = (1, 32, 11, 17)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) # output_shape = (1, 32, 5, 8)
        self.fc1 = nn.Linear(32 * 5 * 8, 16)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(16, 4)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        #x = self.dropout(x)
        
        x = x.reshape(x.shape[0],-1) # output_shape = (1280)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
    def load(self):
        if self.nb_channels == 1:
            self.load_state_dict(torch.load('save_rl/weights_CNN3FC_1channel.pt'))
        else:
            self.load_state_dict(torch.load('save_rl/weights_CNN3FC_2channel.pt'))

    def save(self):
        if self.nb_channels == 1:
            torch.save(self.state_dict(), 'save_rl/weights_CNN3FC_1channel.pt')
        else:
            torch.save(self.state_dict(), 'save_rl/weights_CNN3FC_2channel.pt')

Check that this model fits well the data with a small sanity check.

In [19]:
# Create a test instance of the model
test_model = MultiRegressorCNN3FC(env.observe().shape[3])

# Get a sample observation of the game environment
test_input_tensor = torch.FloatTensor(env.observe())
print(f'Shape of the raw input tensor: {test_input_tensor.shape}')

# Get an output given the sample input and an untrained model just to validate teh correct output size
test_output_tensor = test_model(test_input_tensor)
print(f'Shape of the output tensor: {test_output_tensor.shape}')

Shape of the raw input tensor: torch.Size([1, 29, 41, 1])
Shape of the output tensor: torch.Size([1, 4])


### Initialize Q-Function Approximation Model

Let's **initialize** the neural network of your choice!

Un-comment / comment to choose your network!

In [20]:
# Define model parameters
nb_channels = env.observe().shape[3]
print(f'Number of channels: {nb_channels}')

# Instantiate chosen model and move it to device

# Simple regressor with 1 fully connected layer
#model = MultiRegressor1FC(env.observe()[0], number_of_channels=nb_channels)

# Simple regressor with 2 fully connected layers
model = MultiRegressor2FC(env.observe()[0], number_of_channels=nb_channels)

# CNN regressor with 1 fully connected layer
#model = MultiRegressorCNN1FC(number_of_channels=nb_channels)

# CNN regressor with 2 fully connected layers
#model = MultiRegressorCNN2FC(number_of_channels=nb_channels)

# CNN regressor with 3 fully connected layers
#model = MultiRegressorCNN3FC(number_of_channels=nb_channels)

#model.to(device=device)

model

Number of channels: 1


MultiRegressor2FC(
  (fc1): Linear(in_features=1189, out_features=16, bias=True)
  (selu): SELU()
  (linear): Linear(in_features=16, out_features=4, bias=True)
)

### Loss Function and Optimizer

Define a **loss function** and **optimizer**.

In [21]:
# Define the loss function as cross-entropy
criterion = nn.MSELoss()

# Set stochastic gradient descent as the optimizer
#optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)

# Set Adam as the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Train the model

Training parameters.

In [22]:
number_of_batches = 8  # Number of batches per epoch
batch_size = 32  # Number of experiences we use for training per batch

Reset global maximum reward counter per 100 epoch.

In [23]:
max_reward = 0

In [24]:
max_reward

0

Training routine definition.

In [25]:
def play(model, epochs, criterion, optimizer=None, train=True):

    win_cnt = 0
    lose_cnt = 0
    draw_cnt = 0
    win_hist = []
    cheeses = []
    
    # Addition to track rewards
    reward_cnt = 0
    rewards = []
    
    steps = 0.
    last_W = 0
    last_D = 0
    last_L = 0
    
    global max_cheese
    global max_reward
    
    for e in tqdm(range(epochs)):
        env.reset()
        game_over = False

        # Get the current state of the environment
        state = env.observe()
        
        # Play a full game until game is over
        while not game_over:
            # Do not forget to transform the input of model into torch tensor
            state = torch.FloatTensor(state)

            # Predict the Q value for the current state
            q_values = model(state)

            # Pick the next action that maximizes the Q value
            action = torch.argmax(q_values)
            
            # Apply action, get rewards and new state
            previous_state = state.detach().clone()
            state, reward, game_over= env.act(action)
            
            # Statistics            
            reward_cnt += reward
            
            if game_over:
                steps += env.round
                if env.score > env.enemy_score:
                    win_cnt += 1
                elif env.score == env.enemy_score:
                    draw_cnt += 1
                else:
                    lose_cnt += 1
                cheese = env.score

            # Create an experience array using previous state, the performed action, the obtained reward and the new state. The vector has to be in this order.
            # Store in the experience replay buffer an experience and end game.
            # Do not forget to transform the previous state and the new state into torch tensor.
            # Create an experience array
            experience = [torch.FloatTensor(previous_state), action, reward, torch.FloatTensor(state)]

            # Store the experience in the experience replay buffer
            exp_replay.remember(experience, game_over)
            
        win_hist.append(win_cnt)  # Statistics
        cheeses.append(cheese)  # Statistics
        
        # Save the total reward of this episode and reset the episode reward counter
        rewards.append(reward_cnt)        
        reward_cnt = 0

        if train:

            # Train using experience replay. For each batch, get a set of experiences (state, action, new state) that were stored in the buffer. 
            # Use this batch to train the model.
            
            running_loss = 0
            for b in range(number_of_batches):
                # Get the batch data
                states, Q = exp_replay.get_batch(model, batch_size=batch_size)
                
                # Fit the training data in mounted device
                #states = states.to(device=device)                
                #Q = Q.to(device=device)                
               
                # Compute the loss
                loss = rl.train_on_batch(model, states, Q, criterion, optimizer)
                
                # statistics
                running_loss += loss
            #print('[%d] loss: %.3f' % (e + 1, running_loss / number_of_batches))
            running_loss = 0.0

            '''if e > 100 :  # Check to save
                cheese_np = np.array(cheeses)
                if cheese_np[-100:].sum() > max_cheese:
                    max_cheese = cheese_np[-100:].sum()
                    print(f"New maximum cheese: {max_cheese}.\nSaving model...")
                    model.save()'''
                    
            if e > 100 :  # Check to save
                rewards_np = np.array(rewards)
                if rewards_np[-100:].sum() > max_reward:
                    max_reward = rewards_np[-100:].sum()
                    print(f"New maximum rewards: {max_reward}.\nSaving model...")
                    model.save()   
        
        if (e+1) % 100 == 0:  # Statistics every 100 epochs
            cheese_np = np.array(cheeses)
            rewards_np = np.array(rewards)
            string = "Epoch {:03d}/{:03d} | Last 100 Reward {} | Last 100 Cheese {}| W/D/L {}/{}/{} | 100 W/D/L {}/{}/{} | 100 Steps {}".format(
                        e,epochs, rewards_np[-100:].sum(), 
                        cheese_np[-100:].sum(), win_cnt, draw_cnt, lose_cnt, 
                        win_cnt-last_W, draw_cnt-last_D, lose_cnt-last_L, steps/100)
            print(string)
        
            steps = 0.
            last_W = win_cnt
            last_D = draw_cnt
            last_L = lose_cnt  

### Train the Q-learner with Experience replay

If load, then the last saved result is loaded and training is continued. Otherwise, training is performed from scratch starting with random parameters.

In [26]:
load = False

if load:
    model.load()

Train the model.

In [30]:
epoch = 10000  # Total number of epochs that will be done

print("Training")
play(model, epoch, criterion, optimizer, True)
print("Training done")

#model.save()

Training


  1%|▊                                                                             | 102/10000 [00:07<13:16, 12.43it/s]

Epoch 099/10000 | Last 100 Reward 1983.0 | Last 100 Cheese 1926.5| W/D/L 48/13/39 | 100 W/D/L 48/13/39 | 100 Steps 72.29


  2%|█▌                                                                            | 202/10000 [00:15<12:25, 13.15it/s]

Epoch 199/10000 | Last 100 Reward 1991.0 | Last 100 Cheese 1939.0| W/D/L 102/29/69 | 100 W/D/L 54/16/30 | 100 Steps 71.47


...

Epoch 9799/10000 | Last 100 Reward 2046.0 | Last 100 Cheese 1959.0| W/D/L 5619/1192/2989 | 100 W/D/L 58/14/28 | 100 Steps 71.69


 99%|████████████████████████████████████████████████████████████████████████████▏| 9902/10000 [13:12<00:07, 13.00it/s]

Epoch 9899/10000 | Last 100 Reward 1996.0 | Last 100 Cheese 1945.0| W/D/L 5674/1202/3024 | 100 W/D/L 55/10/35 | 100 Steps 70.79


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [13:20<00:00, 12.49it/s]

Epoch 9999/10000 | Last 100 Reward 2037.0 | Last 100 Cheese 1966.5| W/D/L 5740/1214/3046 | 100 W/D/L 66/12/22 | 100 Steps 68.31
Training done


### Evaluate the Q-learner model

Load the best performant weight parameters.

In [31]:
# Evaluate previous model
load = True

if load:
    model.load()

Evaluate the model.

In [33]:
epoch = 1000  # Total number of epochs that will be done

print("Testing")
play(model, epoch, criterion, optimizer, False)
print("Testing done")

Testing


 11%|████████▉                                                                      | 113/1000 [00:01<00:11, 74.67it/s]

Epoch 099/1000 | Last 100 Reward 2049.0 | Last 100 Cheese 1979.5| W/D/L 61/14/25 | 100 W/D/L 61/14/25 | 100 Steps 71.78


 21%|████████████████▊                                                              | 213/1000 [00:02<00:10, 77.84it/s]

Epoch 199/1000 | Last 100 Reward 2065.0 | Last 100 Cheese 1990.0| W/D/L 124/28/48 | 100 W/D/L 63/14/23 | 100 Steps 71.17


...

Epoch 799/1000 | Last 100 Reward 2066.0 | Last 100 Cheese 1980.0| W/D/L 508/93/199 | 100 W/D/L 66/12/22 | 100 Steps 71.07


 91%|███████████████████████████████████████████████████████████████████████▋       | 907/1000 [00:11<00:01, 78.01it/s]

Epoch 899/1000 | Last 100 Reward 2038.0 | Last 100 Cheese 1964.5| W/D/L 569/102/229 | 100 W/D/L 61/9/30 | 100 Steps 69.03


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 76.79it/s]

Epoch 999/1000 | Last 100 Reward 1986.0 | Last 100 Cheese 1920.5| W/D/L 625/113/262 | 100 W/D/L 56/11/33 | 100 Steps 71.83
Testing done
